# Ableton LLM Control

## Setup

### Imports

In [1]:
import live
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline

### Constants

In [2]:
MODEL_NAME = "ableton_osc_matcher"
MODEL_PATH = f"../artifacts/{MODEL_NAME}"

## Ableton Live connection

In [3]:
livequery = live.Query()
test_response = livequery.query("/live/test")
print(f"Test: {test_response[0]}")

Test: ok


## Text classifier

In [4]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH)
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)

print(classifier)

## Define the pipeline

In [5]:
def classify_text(command: str) -> str:
	results: list[dict] = classifier(command)
	osc_result = results[0]
	print(f"OSC Result: {osc_result}")
	if osc_result["score"] < 0.3:
		raise Exception("Low confidence")

	return osc_result["label"]

In [6]:
def run(command: str) -> None:
	osc_endpoint = classify_text(command)
	live_result = livequery.cmd(osc_endpoint)
	print(f"Live result: {live_result}")

## Run the pipeline

In [12]:
command = ""
run(command)

OSC Result: {'label': '/live/song/stop_playing', 'score': 0.8274032473564148}
Live result: None
